# 🎓 Workshop: Hybrid RAG - Graph + Vector Search with LangChain

## Overview
In this notebook, you'll learn how to build a **Hybrid RAG** system using LangChain that combines:
1. **Graph RAG**: Structured knowledge from Neo4j (relationships, entities)
2. **Vector RAG**: Semantic search using embeddings in Qdrant
3. **Hybrid Retrieval**: Best of both worlds for superior answers

## Why Hybrid RAG?
- **Graph RAG**: Great for relationships, structured queries, multi-hop reasoning
- **Vector RAG**: Excellent for semantic similarity, fuzzy matching
- **Hybrid**: Combines precision of graphs with flexibility of vectors

## Architecture
```
Document → Entity Extraction → Neo4j Graph (GraphCypherQAChain)
        → Chunking → Embeddings → Qdrant Vector DB (Similarity Search)
                                      ↓
Query → Graph Search + Vector Search → Merge Results → LLM Answer
```

---

## 📦 Step 1: Install Dependencies

In [ ]:
!pip install -q neo4j>=5.15.0 qdrant-client>=1.7.0 sentence-transformers>=2.2.2 \
    torch>=2.1.0 langchain>=0.1.0 langchain-community>=0.0.10 langchain-groq>=1.0.0 \
    groq>=0.4.0 python-dotenv>=1.0.0 pydantic>=2.5.0

## 🔧 Step 2: Setup Environment

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

print("✅ Environment variables loaded:")
print(f"  - Neo4j URI: {os.getenv('NEO4J_URI')[:30]}...")
print(f"  - Qdrant URL: {os.getenv('QDRANT_URL')[:30]}...")
print(f"  - Groq API Key: {'✓ Set' if os.getenv('GROQ_API_KEY') else '✗ Missing'}")

## 📚 Step 3: Load Sample Data

In [ ]:
with open('data/samples/university_research_network.md', 'r') as f:
    document_text = f.read()

print(f"📄 Document loaded: {len(document_text)} characters")

## 🤖 Step 4: Initialize LLM

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key=os.getenv('GROQ_API_KEY'),
    model_name="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)

print("✅ LLM initialized (Moonshot AI Kimi K2)")

## 📝 Step 5: Chunk Document for Vector Embeddings

Split document into smaller chunks for better semantic search

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split document into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ". ", " ", ""]
)

chunks = text_splitter.split_text(document_text)

print(f"✅ Created {len(chunks)} chunks")
print(f"\nExample chunk:\n{chunks[0]}")

## 🧮 Step 6: Generate Vector Embeddings

Use sentence-transformers to create embeddings

In [ ]:
from sentence_transformers import SentenceTransformer

# Initialize embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for all chunks
print("🔄 Generating embeddings...")
embeddings = embedding_model.encode(chunks, show_progress_bar=True)

print(f"✅ Generated {len(embeddings)} embeddings")
print(f"   Embedding dimension: {embeddings[0].shape[0]}")

## 💾 Step 7: Store Vectors in Qdrant

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

# Connect to Qdrant
qdrant_client = QdrantClient(
    url=os.getenv('QDRANT_URL'),
    api_key=os.getenv('QDRANT_API_KEY')
)

collection_name = "workshop_chunks"

# Clean up existing collection
print("🧹 Cleaning up existing Qdrant collection...")
try:
    qdrant_client.delete_collection(collection_name)
    print(f"   Deleted existing collection: {collection_name}")
except:
    print(f"   No existing collection found")

print(f"\n📦 Creating fresh collection: {collection_name}")
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)
print("✅ Collection created!")

# Upload vectors
points = [
    PointStruct(
        id=idx,
        vector=embeddings[idx].tolist(),
        payload={"text": chunks[idx]}
    )
    for idx in range(len(chunks))
]

qdrant_client.upsert(collection_name=collection_name, points=points)

print(f"✅ Uploaded {len(points)} vectors to Qdrant")

## 🔗 Step 8: Build Knowledge Graph with LangChain

In [ ]:
from langchain_community.graphs import Neo4jGraph
import json
from groq import Groq

# Connect to Neo4j
graph = Neo4jGraph(
    url=os.getenv('NEO4J_URI'),
    username=os.getenv('NEO4J_USERNAME'),
    password=os.getenv('NEO4J_PASSWORD')
)

print("✅ Connected to Neo4j")

# Clean up existing data
print("\n🧹 Cleaning up existing data...")
count_result = graph.query("MATCH (n) RETURN count(n) as node_count")
node_count = count_result[0]['node_count'] if count_result else 0
print(f"   Found {node_count} existing nodes")
graph.query("MATCH (n) DETACH DELETE n")
print("✅ Database cleaned!")

# Extract entities
groq_client = Groq(api_key=os.getenv('GROQ_API_KEY'))

extraction_prompt = f"""Extract entities from this text. Return a JSON array.
Each entity: name, type (UNIVERSITY, PERSON, RESEARCH_AREA), description.

Text: {document_text}

Return ONLY valid JSON array."""

response = groq_client.chat.completions.create(
    model="moonshotai/kimi-k2-instruct-0905",
    messages=[{"role": "user", "content": extraction_prompt}],
    temperature=0
)

entities = json.loads(response.choices[0].message.content)
print(f"✅ Extracted {len(entities)} entities")

# Extract relationships
relationship_prompt = f"""Extract relationships. Return JSON array.
Each: source, target, type.

Text: {document_text}

Return ONLY valid JSON array."""

response = groq_client.chat.completions.create(
    model="moonshotai/kimi-k2-instruct-0905",
    messages=[{"role": "user", "content": relationship_prompt}],
    temperature=0
)

relationships = json.loads(response.choices[0].message.content)
print(f"✅ Extracted {len(relationships)} relationships")

# Build graph
import hashlib

for entity in entities:
    # Generate unique ID
    entity_id = hashlib.md5(f"{entity['name']}_{entity['type']}".encode()).hexdigest()[:16]
    
    graph.query(
        "MERGE (e:Entity {id: $id, name: $name, type: $type}) SET e.description = $desc",
        params={
            'id': entity_id,
            'name': entity['name'],
            'type': entity['type'],
            'desc': entity.get('description', '')
        }
    )

for rel in relationships:
    try:
        cypher = f"""
        MATCH (s:Entity {{name: $source}})
        MATCH (t:Entity {{name: $target}})
        MERGE (s)-[:{rel['type'].upper().replace(' ', '_')}]->(t)
        """
        graph.query(cypher, params={'source': rel['source'], 'target': rel['target']})
    except:
        pass

print("✅ Knowledge graph built")

## 🔍 Step 9: Vector Search Function

In [ ]:
def vector_search(query: str, top_k: int = 3):
    """Search Qdrant for similar chunks"""
    query_embedding = embedding_model.encode(query)
    
    results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=top_k
    )
    
    return [{
        'text': hit.payload['text'],
        'score': hit.score
    } for hit in results]

# Test vector search
test_results = vector_search("artificial intelligence research")
print("🔍 Vector search test:")
for i, r in enumerate(test_results, 1):
    print(f"\n{i}. Score: {r['score']:.3f}")
    print(f"   {r['text'][:150]}...")

## 🔗 Step 10: Graph Search with GraphCypherQAChain

In [ ]:
from langchain.chains import GraphCypherQAChain

# Create graph QA chain
graph.refresh_schema()

graph_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=False,
    allow_dangerous_requests=True
)

def graph_search(query: str):
    """Search graph using natural language"""
    try:
        result = graph_chain.invoke({"query": query})
        return result['result']
    except Exception as e:
        return f"Graph search error: {str(e)}"

# Test graph search
test_result = graph_search("Tell me about Stanford University")
print("🔗 Graph search test:")
print(test_result)

## 🎯 Step 11: Hybrid RAG - Combine Both!

Merge graph and vector search results for comprehensive answers

In [ ]:
def hybrid_rag(query: str):
    """Hybrid RAG: Combine graph + vector search"""
    print(f"❓ Query: {query}\n")
    
    # 1. Vector search
    print("🔍 Vector Search:")
    vector_results = vector_search(query, top_k=3)
    vector_context = "\n".join([r['text'] for r in vector_results])
    for i, r in enumerate(vector_results, 1):
        print(f"  {i}. {r['text'][:100]}... (score: {r['score']:.3f})")
    
    # 2. Graph search
    print("\n🔗 Graph Search:")
    graph_context = graph_search(query)
    print(f"  {graph_context[:200]}...")
    
    # 3. Combine contexts
    combined_context = f"""Graph Knowledge:
{graph_context}

Vector Search Results:
{vector_context}"""
    
    # 4. Generate answer
    print("\n💡 Generating hybrid answer...")
    response = groq_client.chat.completions.create(
        model="moonshotai/kimi-k2-instruct-0905",
        messages=[
            {"role": "system", "content": "Answer based on the provided context."},
            {"role": "user", "content": f"Context:\n{combined_context}\n\nQuestion: {query}"}
        ],
        temperature=0
    )
    
    answer = response.choices[0].message.content
    print(f"\n✅ Hybrid Answer:\n{answer}")
    return answer

print("✅ Hybrid RAG function ready")

## 🎪 Step 12: Try Different Queries!

In [ ]:
# Query 1: Entity information
hybrid_rag("Tell me about Stanford University")

In [ ]:
# Query 2: Find connections
hybrid_rag("Which entities are connected to Stanford University?")

In [ ]:
# Query 3: List collaborations
hybrid_rag("How many collaborations do we have and list them?")

In [ ]:
# Query 4: List partnerships
hybrid_rag("List all the Partnerships")

## 📊 Step 13: Compare Search Methods

Let's compare vector-only vs graph-only vs hybrid

In [ ]:
test_query = "Tell me about Stanford University"

print("="*60)
print("COMPARISON: Vector vs Graph vs Hybrid")
print("="*60)

# Vector only
print("\n1️⃣ VECTOR SEARCH ONLY:")
print("-" * 60)
vector_results = vector_search(test_query, top_k=2)
for r in vector_results:
    print(f"• {r['text'][:150]}...")

# Graph only
print("\n2️⃣ GRAPH SEARCH ONLY:")
print("-" * 60)
graph_result = graph_search(test_query)
print(graph_result)

# Hybrid
print("\n3️⃣ HYBRID RAG:")
print("-" * 60)
hybrid_rag(test_query)

## 🎯 Key Takeaways

### Vector Search Strengths:
- ✅ Semantic similarity matching
- ✅ Handles synonyms and paraphrasing
- ✅ Good for fuzzy queries
- ✅ Fast retrieval

### Graph Search Strengths:
- ✅ Precise entity matching
- ✅ Relationship traversal
- ✅ Multi-hop reasoning
- ✅ Structured queries

### Hybrid RAG Benefits:
- 🎯 **Best of both worlds**
- 🎯 More comprehensive context
- 🎯 Better answer quality
- 🎯 Handles diverse query types

### LangChain Components Used:
- ✅ `Neo4jGraph`: Graph database connection
- ✅ `GraphCypherQAChain`: Natural language to Cypher
- ✅ `RecursiveCharacterTextSplitter`: Smart text chunking
- ✅ `ChatGroq`: LLM for generation

---

## 🚀 Next Steps

- Experiment with different chunk sizes
- Try other embedding models
- Add more entity types to the graph
- Implement weighted hybrid scoring
- Deploy to production!

---

**Workshop Complete!** 🎉